# Optimization hyperparameter

In [1]:
# Import dependencies 
import pandas as pd
import os
import matplotlib.pyplot as plt

In [2]:
# import dependencies
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [3]:
# Import encoded datafram
encoded_df3 = pd.read_csv('encoded_df3.csv')
encoded_df3

,state_beer,style,availability,abv,types,taste
0,36,German Hefeweizen,2,5.0,4,4.50
1,36,German Hefeweizen,6,5.6,2,4.50
2,43,German Hefeweizen,6,5.4,1,4.00
3,37,German Hefeweizen,4,5.0,10,4.25
4,46,German Hefeweizen,2,4.5,3,4.00
...,...,...,...,...,...,...
443995,4,Stout,6,5.8,3,4.50
443996,10,Stout,2,8.0,2,3.50
443997,4,Stout,2,11.0,10,5.00
443998,22,Stout,2,13.1,5,4.75


In [3]:
#encode string values
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
encoded_df4 = encoded_df3.copy()
encoded_df4['style'] = le.fit_transform(encoded_df4['style'])
encoded_df4

,state_beer,style,availability,abv,types,taste
0,36,18,2,5.0,4,4.50
1,36,18,6,5.6,2,4.50
2,43,18,6,5.4,1,4.00
3,37,18,4,5.0,10,4.25
4,46,18,2,4.5,3,4.00
...,...,...,...,...,...,...
443995,4,36,6,5.8,3,4.50
443996,10,36,2,8.0,2,3.50
443997,4,36,2,11.0,10,5.00
443998,22,36,2,13.1,5,4.75


In [4]:
# Define the target set
y = encoded_df3["style"].values
y

array(['German Hefeweizen', 'German Hefeweizen', 'German Hefeweizen', ...,
       'Stout', 'Stout', 'Stout'], dtype=object)

In [5]:
#define the features set
X = encoded_df3.copy()
X = X.drop(columns =["style"])
X

,state_beer,availability,abv,types,taste
0,36,2,5.0,4,4.50
1,36,6,5.6,2,4.50
2,43,6,5.4,1,4.00
3,37,4,5.0,10,4.25
4,46,2,4.5,3,4.00
...,...,...,...,...,...
443995,4,6,5.8,3,4.50
443996,10,2,8.0,2,3.50
443997,4,2,11.0,10,5.00
443998,22,2,13.1,5,4.75


In [6]:
# Splitting into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)

In [7]:
# Create a StandardScaler instance
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [8]:
best_trial1 = {'lambda': 0.6961487559906326, 'alpha': 0.010396164166391456, 'colsample_bytree': 0.9, 'subsample': 0.7, 'learning_rate': 0.008, 'max_depth': 17, 'random_state': 2020, 'min_child_weight': 1,'n_estimators': 100 }
best_trial1

{'lambda': 0.6961487559906326,
 'alpha': 0.010396164166391456,
 'colsample_bytree': 0.9,
 'subsample': 0.7,
 'learning_rate': 0.008,
 'max_depth': 17,
 'random_state': 2020,
 'min_child_weight': 1,
 'n_estimators': 100}

In [9]:
# Create a random forest classifier
xgb_model = XGBClassifier(**best_trial1)
# fit the model
xgb_model = xgb_model.fit(X_train_scaled, y_train)

/Users/alexzzf/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[13:44:47] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1645117948562/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [10]:
# make predictions for test data
predictions = xgb_model.predict(X_test_scaled)

In [11]:
#calculate matrix and accuracy score
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(cm)
acc_score = accuracy_score(y_test, predictions)


In [12]:
display(cm_df)

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,2789,4,12,1,8,18,44,52,17,21,...,22,12,22,13,2,50,8,1,32,1
1,2,2990,7,185,61,0,0,0,3,29,...,7,6,2,2,18,6,103,76,19,5
2,10,10,3055,11,31,2,3,7,10,29,...,12,9,23,48,29,9,9,3,49,16
3,4,85,1,3181,37,1,0,0,4,9,...,6,1,0,4,18,5,60,49,4,0
4,11,39,19,44,3084,4,1,0,11,42,...,27,9,0,9,43,0,23,30,14,4
5,57,6,20,2,21,2423,43,64,81,13,...,19,5,18,36,3,5,10,1,42,10
6,33,0,13,0,0,8,3000,43,9,4,...,8,0,5,6,0,13,0,0,83,0
7,56,0,7,0,4,88,74,2423,38,31,...,22,36,29,23,7,30,1,1,55,4
8,62,20,42,11,24,16,38,48,2393,46,...,45,21,24,18,17,18,52,29,51,28
9,9,80,59,49,115,13,60,26,27,2295,...,32,16,23,34,67,23,58,71,70,46


In [13]:
# Display results
print("Confusion Matrix")
print(f"Accuracy Score: {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix
Accuracy Score: 0.751064701064701
Classification Report
                                             precision    recall  f1-score   support

                         American Cream Ale       0.72      0.76      0.74      3651
                                 Barleywine       0.76      0.81      0.79      3689
                             Belgian Dubbel       0.78      0.83      0.81      3668
                   Belgian Quadrupel (Quad)       0.83      0.87      0.85      3638
                             Belgian Tripel       0.71      0.86      0.78      3584
                            Belgian Witbier       0.77      0.66      0.71      3683
                            Berliner Weisse       0.68      0.81      0.74      3681
                                 Blonde Ale       0.69      0.66      0.67      3685
                                  Brown Ale       0.70      0.65      0.68      3671
                                   Dark Ale       0.69      0.61      0.65  

In [14]:
# calculate feature importance in the Random Forest model
importances = xgb_model.feature_importances_
importances

array([0.17993784, 0.30919743, 0.30770978, 0.1774549 , 0.02570013],
      dtype=float32)

In [15]:
# Sort the features by importances
sorted(zip(xgb_model.feature_importances_, X.columns), reverse = True)

[(0.30919743, 'availability'),
 (0.30770978, 'abv'),
 (0.17993784, 'state_beer'),
 (0.1774549, 'types'),
 (0.025700128, 'taste')]

In [26]:
#Create result dataframe
trial_report = classification_report(y_test, predictions, output_dict = True)
report_df = pd.DataFrame(trial_report).transpose()
report_df

,precision,recall,f1-score,support
American Cream Ale,0.724039,0.763900,0.743436,3651.000000
Barleywine,0.762950,0.810518,0.786015,3689.000000
Belgian Dubbel,0.783133,0.832879,0.807240,3668.000000
Belgian Quadrupel (Quad),0.831199,0.874382,0.852244,3638.000000
Belgian Tripel,0.707502,0.860491,0.776533,3584.000000
Belgian Witbier,0.769206,0.657888,0.709205,3683.000000
Berliner Weisse,0.684151,0.814996,0.743863,3681.000000
Blonde Ale,0.687961,0.657531,0.672402,3685.000000
Brown Ale,0.700322,0.651866,0.675226,3671.000000
Dark Ale,0.689603,0.612490,0.648763,3747.000000


In [28]:
#save csv file
report_df.to_csv('report.csv',index = True)